In [1]:
pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25855 sha256=43685ee228cbb0fac9c6d0d85833eb971ec50e2cf4633e63efeeae050672c1b4
  Stored in directory: c:\users\aravi\appdata\local\pip\cache\wheels\85\cf\3a\e265e975a1e7c7e54eb3692d6aa4e2e7d6a3945d29da46f2d7
Successfully built imutils
Note: you may need to restart the kernel to use updated packages.


In [3]:
from ultralytics import YOLO
import tensorflow as tf
import cv2
import os
import imutils
import cv2
import itertools, os, time
import numpy as np
import argparse
import pandas as pd
import operator
import os, shutil


Build model

In [4]:
import string

# Original mapping from class_name to English letters
original_mapping = {
    "1": "1", "6": "6", "5": "5", "4": "4", "7": "7", "9": "9", "3": "3", "2": "2", "8": "8",
    "meem": "م", "0": "0", "raa": "ر", "seen": "س", "alif": "ا", "daal": "د", "noon": "ن",
    "waw": "و", "ain": "ع", "haa": "ه", "laam": "ل", "jeem": "ج", "baa": "ب", "qaaf": "ق",
    "yaa": "ى", "faa": "ف", "Taa": "ط", "saad": "ص", "thaa": "ث", "ghayn": "ج", "sheen": "ش",
    "Thaa": "ظ", "khaa": "خ", "kaaf": "ك", "taa": "ت", "zay": "ز", "zaal": "ذ", "7aa": "ح",
    "daad": "ض"
}

# Arabic to English mapping
ar_to_en = {
    "١": "1", "٢": "2", "٣": "3", "٤": "4", "٥": "5", "٦": "6", "٧": "7", "٨": "8", "٩": "9", "٠": "0",
    "ا": "A", "ب": "B", "ت": "C", "ث": "D", "ج": "E", "ح": "F", "خ": "G", "د": "H", "ذ": "I",
    "ر": "J", "ز": "K", "س": "L", "ش": "M", "ص": "N", "ض": "O", "ط": "P", "ظ": "Q", "ع": "R", "غ": "S",
    "ف": "T", "ق": "U", "ك": "V", "ل": "W", "م": "n", "ن": "Y", "ه": "Z", "و": "a", "ى": "b",
    "1": "1", "2": "2", "3": "3", "4": "4", "5": "5", "6": "6", "7": "7", "8": "8", "9": "9", "0": "0"
}

en_to_ar = {v: k for k, v in ar_to_en.items()}

# Remove duplicate values to get unique letters
unique_letters = sorted(set(ar_to_en.values()))

# Add 'X' as the padding character
unique_letters.append('X')

# Convert the unique letters list to a string
CHAR_VECTOR = ''.join(unique_letters)

# Create a list of letters
letters = [letter for letter in CHAR_VECTOR]

num_classes = len(letters) + 1

img_w, img_h = 128, 64

# Network parameters
batch_size = 8
val_batch_size = 1

downsample_factor = 4
max_text_len = 7

print(CHAR_VECTOR)

0123456789ABCDEFGHIJKLMNOPQRSTUVWYZabnX


Model Layers

In [5]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Input
from tensorflow.keras.layers import BatchNormalization, Reshape, Lambda
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.layers import add, concatenate


# VGG Blocks
class VggBlock1(keras.layers.Layer):
    def __init__(self, num_filters, do_maxpool: bool = True, **kwargs):
        super(VggBlock1, self).__init__(**kwargs)
        self.num_filters = num_filters
        self.do_maxpool = do_maxpool
        self.conv = Conv2D(
            num_filters, (3, 3), padding="same", kernel_initializer="he_normal"
        )
        self.bn = BatchNormalization()
        self.relu = Activation("relu")
        if self.do_maxpool:
            self.pool = MaxPooling2D(pool_size=(2, 2))

    def call(self, inputs):
        x = self.conv(inputs)
        x = self.bn(x)
        x = self.relu(x)
        if self.do_maxpool:
            x = self.pool(x)
        return x

    def get_config(self):
        config = super(VggBlock1, self).get_config()
        config.update({
            'num_filters': self.num_filters,
            'do_maxpool': self.do_maxpool
        })
        return config


class VggBlock2(keras.layers.Layer):
    def __init__(self, num_filters, **kwargs):
        super(VggBlock2, self).__init__(**kwargs)
        self.num_filters = num_filters
        
        self.conv1 = Conv2D(
            num_filters, (3, 3), padding="same", kernel_initializer="he_normal"
        )
        self.bn1 = BatchNormalization()
        self.relu1 = Activation("relu")
        self.conv2 = Conv2D(
            num_filters, (3, 3), padding="same", kernel_initializer="he_normal"
        )
        self.bn2 = BatchNormalization()
        self.relu2 = Activation("relu")
        self.pool = MaxPooling2D(pool_size=(1, 2))

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        x = self.pool(x)
        return x
    
    def get_config(self):
        config = super(VggBlock2, self).get_config()
        config.update({"num_filters": self.num_filters})
        return config



# RNN
class BI_LSTM_Block(keras.layers.Layer):
    def __init__(self, num_units, merge_mode, **kwargs):
        super(BI_LSTM_Block, self).__init__(**kwargs)
        self.num_units = num_units
        self.merge_mode = merge_mode

        self.bi_lstm = Bidirectional(
            LSTM(num_units, return_sequences=True, kernel_initializer="he_normal"),
            merge_mode=merge_mode,
        )
        self.bn = BatchNormalization()

    def call(self, inputs):
        x = self.bi_lstm(inputs)
        x = self.bn(x)
        return x

    def get_config(self):
        config = super(BI_LSTM_Block, self).get_config()
        config.update({"num_units": self.num_units, "merge_mode": self.merge_mode})
        return config



def ctc_lamda_func(args):
    y_pred, labels, input_length, label_length = args

    y_pred = y_pred[:, 2:, :]
    return keras.backend.ctc_batch_cost(labels, y_pred, input_length, label_length)


def build_model(training):

    input_shape = (128, 64, 1)

    inputs = Input(name="the_input", shape=input_shape, dtype="float32")

    # VGG (CNN)
    x = VggBlock1(64, name="VGG_Block1")(inputs)
    x = VggBlock1(128, name="VGG_Block2")(x)
    x = VggBlock2(256, name="VGG_Block3")(x)
    x = VggBlock2(512, name="VGG_Block4")(x)
    x = VggBlock1(512, do_maxpool=False, name="VGG_Block5")(x)

    # CNN to RNN
    new_shape = (32, 2048)
    x = Reshape(target_shape=new_shape, name="reshape")(x)
    x = Dense(64, activation="relu", kernel_initializer="he_normal", name="dense1")(x)

    # RNN
    x = BI_LSTM_Block(256, merge_mode="sum", name="BI_LSTM_Block1")(x)
    x = BI_LSTM_Block(128, merge_mode="concat", name="BI_LSTM_Block2")(x)

    x = Dropout(0.25)(x)

    # RNN output to character activations
    x = Dense(num_classes, kernel_initializer="he_normal", name="dense2")(x)
    y_pred = Activation("softmax", name="softmax")(x)

    labels = Input(name="the_labels", shape=[max_text_len], dtype="float32")
    input_length = Input(name="input_length", shape=[1], dtype="int64")
    label_length = Input(name="label_length", shape=[1], dtype="int64")

    loss_out = Lambda(ctc_lamda_func, output_shape=(1,), name="ctc")(
        [y_pred, labels, input_length, label_length]
    )

    if training:
        return keras.Model(
            inputs=[inputs, labels, input_length, label_length], outputs=loss_out
        )
    else:
        return keras.Model(inputs=[inputs], outputs=y_pred)

Model Checkpoints

In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Define the EarlyStopping and ModelCheckpoint callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=25, verbose=1, mode='min', restore_best_weights=True)
model_checkpoint = ModelCheckpoint('model_ad_best.keras', monitor='val_loss', mode='min', save_best_only=True, verbose=1)

In [17]:
model = build_model(True)
opt = tf.keras.optimizers.Adadelta(1.0)
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, metrics=['accuracy'], optimizer=opt)

In [18]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ the_input           │ (None, 128, 64,   │          0 │ -                 │
│ (InputLayer)        │ 1)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VGG_Block1          │ (None, 64, 32,    │        896 │ the_input[0][0]   │
│ (VggBlock1)         │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VGG_Block2          │ (None, 32, 16,    │     74,368 │ VGG_Block1[0][0]  │
│ (VggBlock1)         │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VGG_Block3          │ (None, 32, 8,     │    887,296 │ VGG_Block2[0][0]  │
│ (VggBlock2)         │ 256)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VGG_Block4          │ (None, 32, 4,     │  3,544,064 │ VGG_Block3[0][0]  │
│ (VggBlock2)         │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ VGG_Block5          │ (None, 32, 4,     │  2,361,856 │ VGG_Block4[0][0]  │
│ (VggBlock1)         │ 512)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 32, 2048)  │          0 │ VGG_Block5[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense1 (Dense)      │ (None, 32, 64)    │    131,136 │ reshape[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ BI_LSTM_Block1      │ (None, 32, 256)   │    658,432 │ dense1[0][0]      │
│ (BI_LSTM_Block)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ BI_LSTM_Block2      │ (None, 32, 256)   │    395,264 │ BI_LSTM_Block1[0… │
│ (BI_LSTM_Block)     │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 32, 256)   │          0 │ BI_LSTM_Block2[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense2 (Dense)      │ (None, 32, 40)    │     10,280 │ dropout[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ softmax             │ (None, 32, 40)    │          0 │ dense2[0][0]      │
│ (Activation)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ the_labels          │ (None, 7)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_length        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ label_length        │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ ctc (Lambda)        │ (None, 1)         │          0 │ softmax[0][0],    │
│                     │                   │            │ the_labels[0][0], │
│                     │                   │            │ input_length[0][… │
│                     │                   │            │ label_length[0][… │
└─────────────────────┴───────────────────┴────────────┴─────────────────

 Total params: 8,063,592 (30.76 MB)

 Trainable params: 8,058,088 (30.74 MB)

 Non-trainable params: 5,504 (21.50 KB)

Encoding and Decoding function

In [7]:
#map the arabic op to english letter
ar_to_plate_map = {
    "ا":"A", "ب":"B", "ح":"J", "د":"D", 
    "ر":"R", "س":"S", "ص":"X/C", "ط":"T", 
    "ع":"E", "ق":"G/K", "ك":"K", "ل":"L", 
    "م":"Z/M", "ن":"N", "هـ":"H", "و":"U/W", "ى":"V", "ج":"G", "ف":"F", "ی":"Y"
}



In [8]:
# Function to convert label to English
def label_to_en(label):
    new = ""
    for l in label:
        if l == " ":
            continue
        elif l in string.digits:
            new += l
        else:
            try:
                new = new + ar_to_plate_map[l]
            except:
                continue
    max_letters = 7
    missing = max_letters - len(new)
    if missing > 0:
        new = new + "-" * missing
    return new

# Function to reverse a string
def reverse_str(s):
    return s[::-1]

# Function to convert label to Arabic
def label_to_ar(label):
    label = reverse_str(label)
    ar_label = ""
    for i in label:
        if i == "X":
            continue
        else:
            ar_label += en_to_ar[i] + " "
    ar_label = ar_label.strip()
    return ar_label

# Function to get label from indexes
def get_label(idxes):
    label = ""
    for idx in idxes:
        if idx < len(letters):
            label += letters[idx]
    return label

# Function to decode label
def decode_label(out):
    out_best = list(np.argmax(out[0, 2:], axis=1))
    out_best = [k for k, g in itertools.groupby(out_best)]
    outstr = ''
    for i in out_best:
        if i < len(letters):
            outstr += letters[i]
    return outstr

In [9]:
numplate_model = build_model(False)
numplate_model.load_weights("models\\best_weight.h5")

In [10]:
#multi pred test

import os
import cv2
from ultralytics import YOLO
import arabic_reshaper
from bidi.algorithm import get_display
import numpy as np
from PIL import ImageFont, ImageDraw, Image

def numberplate_detection(image_path, yolo_model, model):
    img = cv2.imread(image_path)
    results = yolo_model.predict(image_path)
    bounding_boxes = []
    
    # if detections were made
    if results[0].boxes is not None:
        for box in results[0].boxes.xyxy:
            x1, y1, x2, y2 = map(int, box[:4])  # Get bounding box coordinates
            bounding_boxes.append((x1, y1, x2, y2))
    
    labels = []

    for box in bounding_boxes:
        x1, y1, x2, y2 = box
        crop_img = img[y1:y2, x1:x2]  # Crop the image
        #converting to bw
        img_grey = cv2.cvtColor(crop_img, cv2.COLOR_BGR2GRAY)
        
        #process the img
        img_pred = img_grey.astype(np.float32)
        img_pred = cv2.resize(img_pred, (128, 64))
        img_pred = (img_pred / 255.0)
        img_pred = img_pred.T
        img_pred = np.expand_dims(img_pred, axis=-1)
        img_pred = np.expand_dims(img_pred, axis=0)

        #predict on img
        net_out_value = model.predict(img_pred, verbose=1)
        pred_text = decode_label(net_out_value)
        pred_text_ar = label_to_ar(pred_text)

        labels.append(pred_text_ar)

    print(labels)
    return bounding_boxes, labels, img



        

def bbox_and_labels(image, bbox, labels, save_dir, filename):
    img_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(img_pil)

    font = ImageFont.truetype(font= "arial.ttf", size=20)

    for box, label in zip(bbox,labels):
        x1, y1, x2, y2 = box

        #bbox
        draw.rectangle([x1, y1, x2, y2], outline=(0,255,0), width=2)

        #arabic text label
        reshaped_ar_label = arabic_reshaper.reshape(label)
        bidi_ar_text = get_display(reshaped_ar_label)
        text_x1, text_y1, text_x2, text_y2 = draw.textbbox((x1,y1),bidi_ar_text, font=font)         #get the text size

        height = (text_y2 - text_y1) + 5
        width = (text_x2 - text_x1) + 5

        draw.rectangle([(x1, y1-height), (x1+width, y1)], (0,255,0), width=2)
        draw.text((x1+3 , y1-height), bidi_ar_text, font=font, fill=(0, 0, 0))

        #english label
        reshaped_en_label = label_to_en(label)
        bidi_en_text = get_display(reshaped_en_label)
        text1_x1, text1_y1, text1_x2, text1_y2 = draw.textbbox((x1,y1),bidi_en_text, font=font)

        height1 = (text1_y2 - text1_y1) + 5
        width1 = (text1_x2 - text1_x1) + 5

        draw.rectangle([(x1, y2), (x1+width1, y2+height1)], (0,255,0), width=2)
        draw.text((x1+3 , y2), bidi_en_text, font=font, fill=(0, 0, 0))

    img = cv2.cvtColor(np.array(img_pil), cv2.COLOR_RGB2BGR)

    #os.makedirs(save_dir, exist_ok=True)

    # Save the image with bounding boxes and labels
    save_path = os.path.join(save_dir, os.path.basename(filename))
    cv2.imwrite(save_path, img)

    return save_path, img


def save_pred(image_name, bbox, labels, save_dir):

    txt_folder = os.path.join(save_dir, "labels")
    os.makedirs(txt_folder, exist_ok=True)
    txt_file = os.path.join(txt_folder, f"{image_name}.txt")

    with open (txt_file, 'w') as txt:
        for box, label in zip(bbox, labels):
            x1, y1, x2, y2 = box
            en_label = label_to_en(label)
            txt.write(f"Label : {label}, Eng_label : {en_label}, Coordinates : {x1} {y1} {x2} {y2}")
    return txt_file



Predictions

In [17]:
#multiple pred test


def numberplate_recognition(source):

    start = time.time()

    model_path = "models/yv8.pt"
    save_dir = "runs"
    model = numplate_model

    yolo_model = YOLO(model_path)
    os.makedirs(save_dir, exist_ok=True)
    
    if os.path.isdir(source):
        # Iterate over all files
        for filename in os.listdir(source):
            image_path = os.path.join(source, filename)
            if os.path.isfile(image_path) and filename.lower().endswith(('.png', '.jpg', '.jpeg')):

                bounding_boxes, labels, img = numberplate_detection(image_path,yolo_model,model)
                fname = os.path.splitext(filename)[0]
                savepath, img = bbox_and_labels(img,bounding_boxes,labels,save_dir,filename)

                resized = imutils.resize(img, width=640)
                cv2.imshow("output figure",resized)
                cv2.waitKey(0)
                
                textfile = save_pred(fname,bounding_boxes,labels,save_dir)
                print(f"predicted to {savepath} \npredictions saved to {textfile}")
                
    elif os.path.isfile(source):
        bounding_boxes, labels, img = numberplate_detection(source,yolo_model,model)
        fname = os.path.basename(source)
        filename = os.path.splitext(fname)[0]
        savepath, img = bbox_and_labels(img,bounding_boxes,labels,save_dir,fname)
        resized = imutils.resize(img, width=640)
        cv2.imshow("output figure",resized)
        cv2.waitKey(0)
        textfile = save_pred(filename,bounding_boxes,labels,save_dir)
        print(f"predicted to {savepath} \npredictions saved to {textfile}")

    else:
        print(f"Source path '{source}' is neither a valid directory nor a file.")
    cv2.destroyAllWindows()
    end = time.time()
    print(f"Time taken: {end - start:.2f} seconds")
    

In [18]:
test_set = "data/tset"

numberplate_recognition(test_set)


image 1/1 x:\DL\Numberplate\data\tset\0034.jpg: 640x640 1 LicensePlate, 113.3ms
Speed: 8.2ms preprocess, 113.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
['د ط 1']
predicted to runs\0034.jpg 
predictions saved to runs\labels\0034.txt

image 1/1 x:\DL\Numberplate\data\tset\0036.jpg: 640x480 1 LicensePlate, 87.3ms
Speed: 0.0ms preprocess, 87.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 480)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
['د ف ق 5 3 4 9']
predicted to runs\0036.jpg 
predictions saved to runs\labels\0036.txt

image 1/1 x:\DL\Numberplate\data\tset\0038.jpg: 640x640 1 LicensePlate, 100.2ms
Speed: 4.8ms preprocess, 100.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
['س 2']
predicted to runs\0038.jpg 
predictions saved to runs\labels\0038.txt

image 1/1 x:\DL\Numberplate\data\tset\0074.jpg: 640x640 1 LicensePlate, 98.8ms
Speed: 14.9ms preprocess,